### LangChain-Based Stock Analysis Agent

- **IMPORT MODULES**: LOADS REQUIRED MODULES FOR ENVIRONMENT SETUP, DATA HANDLING, AND AI INTEGRATION.  
- **SET API KEY**: STORES THE OPENAI API KEY IN AN ENVIRONMENT VARIABLE.  
- **DEFINE STOCK LIST**: CONTAINS 50 REAL-WORLD STOCK SYMBOLS.  
- **GENERATE STOCK DATA**: CREATES RANDOMIZED STOCK MARKET DATA FOR 10 DAYS.  
- **CREATE DATAFRAME**: STORES STOCK DATA IN A PANDAS DATAFRAME.  
- **FETCH STOCK DATA**: FUNCTION RETURNS HISTORICAL DATA FOR A GIVEN STOCK.  
- **ANALYZE STOCK TREND**: COMPUTES A SIMPLE MOVING AVERAGE (SMA) AND GENERATES A BUY/SELL/HOLD SIGNAL.  
- **DEFINE LANGCHAIN TOOLS**:  
  - TOOL TO FETCH STOCK DATA.  
  - TOOL TO ANALYZE STOCK TRENDS.  
- **INITIALIZE OPENAI LLM**: CREATES AN AI MODEL INSTANCE WITH A SET TEMPERATURE.  
- **CREATE AI AGENT**: INTEGRATES THE LLM WITH THE DEFINED TOOLS USING LANGCHAIN.  
- **EXECUTE QUERY**: RUNS AN AGENT COMMAND TO ANALYZE STOCK TRENDS FOR A SPECIFIC SYMBOL (E.G., AAPL).

In [4]:
# Read the open ai API key from your text file
f = open('C:\\Users\\Shailendra Kadre\\Desktop\\OPEN_AI_KEY.txt')
api_key = f.read()

In [3]:
import os  # Import the os module to set environment variables
import random  # Import random module for generating dummy stock prices
from datetime import datetime, timedelta  # Import datetime for handling dates
from langchain.llms import OpenAI  # Import OpenAI LLM from LangChain
from langchain.agents import initialize_agent, AgentType  # Import agent tools from LangChain
from langchain.tools import Tool  # Import Tool class to create custom LangChain tools
import pandas as pd  # Import pandas for data handling and analysis

# Set up your OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = api_key

# List of 50 real-world stock symbols
stocks = [
    "AAPL", "GOOGL", "AMZN", "TSLA", "MSFT", "NFLX", "META", "NVDA", "BABA", "DIS",
    "V", "JPM", "PYPL", "MA", "INTC", "IBM", "ORCL", "CSCO", "ADBE", "AMD",
    "UBER", "LYFT", "SQ", "SHOP", "TWTR", "SNAP", "PINS", "ZM", "DOCU", "ROKU",
    "BA", "GE", "CAT", "MMM", "F", "GM", "NKE", "KO", "PEP", "MCD",
    "WMT", "TGT", "HD", "LOW", "COST", "PG", "JNJ", "MRNA", "PFE", "BMY"
]

# Generate dummy stock market data
num_days = 10  # Number of days per stock
start_date = datetime(2024, 1, 1)  # Start date
data = []  # Initialize empty list to store stock data

# Generate data for each stock over 10 days
for stock in stocks:
    for i in range(num_days):
        date = start_date + timedelta(days=i)  # Increment date
        open_price = round(random.uniform(100, 1000), 2)  # Random open price
        high_price = round(open_price + random.uniform(5, 50), 2)  # High slightly above open
        low_price = round(open_price - random.uniform(5, 50), 2)  # Low slightly below open
        close_price = round(random.uniform(low_price, high_price), 2)  # Close within range
        volume = random.randint(500000, 50000000)  # Random trading volume
        
        # This line adds a new row to a list called data
        data.append([date.strftime("%Y-%m-%d"), stock, open_price, high_price, low_price, close_price, volume])
        # date.strftime("%Y-%m-%d") - Formats the date object into a YYYY-MM-DD string.

# Create DataFrame
df = pd.DataFrame(data, columns=["Date", "Stock", "Open", "High", "Low", "Close", "Volume"])

# Function to get stock data for a specific symbol
def get_stock_data(symbol: str):
    return df[df["Stock"] == symbol].reset_index(drop=True)  
    # Filter DataFrame for the given stock symbol

# Function to analyze stock trends based on SMA (Simple Moving Average)
def analyze_trend(stock_data):
    stock_data["SMA"] = stock_data["Close"].rolling(window=3).mean()  # Calculate Simple Moving Average (SMA)
    latest_price = stock_data["Close"].iloc[-1]  # Get the latest closing price
    latest_sma = stock_data["SMA"].iloc[-1]  # Get the latest SMA value
    
    """ 
    `latest_price = stock_data["Close"].iloc[-1]`  

    ### Explanation:  
    - `stock_data["Close"]`: Extracts the "Close" column from the `stock_data` DataFrame, which contains closing prices of the stock.  
    - `.iloc[-1]`: Selects the last row of the "Close" column, retrieving the most recent closing price.  

    ### Purpose:  
    This line gets the latest closing price of a stock from the given dataset.
    """

    # Generate trading signal based on SMA comparison
    if latest_price > latest_sma:
        return "Buy signal: Price is above SMA"
    elif latest_price < latest_sma:
        return "Sell signal: Price is below SMA"
    else:
        return "Hold signal: Price is at SMA"

# Define a LangChain tool to fetch stock data
stock_data_tool = Tool(
    name="Stock Data Fetcher",  # Name of the tool
    func=lambda symbol: get_stock_data(symbol).to_string(),  # Function to execute
    description="Fetches dummy stock data for a given symbol"  # Description of the tool
)

# Define a LangChain tool to analyze the stock trend
trend_analysis_tool = Tool(
    name="Stock Trend Analyzer",  # Name of the tool
    func=lambda symbol: analyze_trend(get_stock_data(symbol)),  # Function to execute
    description="Analyzes stock trend and gives buy/sell signals"  # Description of the tool
)

# Initialize the LangChain LLM
llm = OpenAI(api_key=os.environ["OPENAI_API_KEY"], temperature=0.7)  # Create an instance of OpenAI model with moderate randomness

# Create the AI agent using LangChain
agent = initialize_agent(
    tools=[stock_data_tool, trend_analysis_tool],  # List of tools the agent can use
    llm=llm,  # Assign the LLM to the agent
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Specify the type of agent
    verbose=True  # Enable verbose output for debugging
)

# Example usage (replace with actual stock symbol)
response = agent.run("Analyze the trend for AAPL")  # Run the agent with a query
print(response)  # Print the agent's response



> Entering new AgentExecutor chain...
 I should use the Stock Data Fetcher to get the stock data for AAPL
Action: Stock Data Fetcher
Action Input: AAPL
Observation:          Date Stock    Open    High     Low   Close    Volume
0  2024-01-01  AAPL  950.12  957.85  900.64  901.40  15083224
1  2024-01-02  AAPL  525.51  549.66  477.65  518.36  14434448
2  2024-01-03  AAPL  907.62  919.88  863.87  898.99  15286254
3  2024-01-04  AAPL  857.77  875.54  849.98  857.14  35041104
4  2024-01-05  AAPL  941.17  974.03  924.63  934.36  33134796
5  2024-01-06  AAPL  552.19  565.87  529.39  546.66   2792180
6  2024-01-07  AAPL  953.96  960.45  943.50  958.87  41613530
7  2024-01-08  AAPL  482.18  525.64  469.70  485.87  14577853
8  2024-01-09  AAPL  341.44  369.70  319.75  367.96  23713339
9  2024-01-10  AAPL  546.23  564.76  536.35  550.44  27605642
Thought: Now that I have the stock data, I can use the Stock Trend Analyzer to analyze the trend for AAPL
Action: Stock Trend Analyzer
Action Input: AA

### **ROLE OF LLM AND AGENTS IN THIS CODE**  

#### **1. Role of LLM (Large Language Model)**
- `llm = OpenAI(api_key=os.environ["OPENAI_API_KEY"], temperature=0.7)`:  
  - This initializes the OpenAI language model (LLM) using LangChain.  
  - The model is used to process user queries, interpret them, and generate responses.  
  - The `temperature=0.7` allows for a balanced mix of deterministic and creative responses.

#### **2. Role of Agents**
- `agent = initialize_agent(...)`:  
  - The agent acts as an AI-powered assistant that interacts with users and selects the right tools to answer their queries.  
  - It uses `ZERO_SHOT_REACT_DESCRIPTION`, meaning it decides dynamically which tools to use based on the query.  
  - The agent can:
  1. **Fetch stock data** using `stock_data_tool`.  
  2. **Analyze stock trends** using `trend_analysis_tool`.  

#### **Final Workflow**  
- When a user asks: **"Analyze the trend for AAPL"**, the agent:  
  1. **Identifies** that stock analysis is needed.  
  2. **Fetches AAPL stock data** using `stock_data_tool`.  
  3. **Runs trend analysis** using `trend_analysis_tool`.  
  4. **Generates a response** (e.g., "Buy signal: Price is above SMA").